In [36]:
!pip install ffmpeg-python
!pip install -U triton
!pip install -U transformers -U torch
!pip install yt-dlp ffmpeg-python pydub
!pip install youtube-transcript-api
!pip install azure-core==1.29.7
!pip install azure-ai-openai==1.0.0
!pip install openai==1.12.0

ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton


In [50]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from youtube_transcript_api.formatters import TextFormatter
from openai import AzureOpenAI

def get_youtube_transcript_auto_generated(video_id, video_url=None):
      # Step 1: Fetch the available transcripts for the video
      print("Fetching available transcripts for the video...")
      available_transcripts = YouTubeTranscriptApi.list_transcripts(video_id)
      # Step 2: Check if manually created transcripts are available
      transcript = None
      for transcript_obj in available_transcripts:
          print(transcript_obj.language, transcript_obj.is_generated, transcript_obj.is_translatable)
          
          if not transcript:
              if transcript_obj.is_generated:
                print(f"Auto-generated transcript found, language: {transcript_obj.language_code}")
                try:
                    transcript = transcript_obj.fetch()  # Fetch auto-generated transcript
                except:
                    print(f"Could not fetch auto-generated transcript for {transcript_obj.language_code}")
              else:
                print(f"Manually created transcript found, language: {transcript_obj.language_code}")
                try:
                    transcript = transcript_obj.fetch()  # Fetch manually created transcript if available
                except:
                    print(f"Could not fetch manually created transcript for {transcript_obj.language_code}")

          
          if transcript:
              break  # Exit loop once a transcript is found

      if not transcript:
          print("No transcript found for any available languages.")
          return None

      # Step 3: Format the transcript as text
      formatter = TextFormatter()
      formatted_transcript = formatter.format_transcript(transcript)
      return formatted_transcript

def split_text(text, chunk_size=8000):
    """Split text into chunks of approximately chunk_size characters."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        current_chunk.append(word)
        current_length += len(word) + 1  # +1 for space
        
        if current_length >= chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_length = 0
    
    # Add any remaining text as the last chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks


def get_video_summary(transcript, client, chunk_size=8000):
    """Generate a summary of a video transcript file."""
    # Load the transcript
    if not transcript:
        return "Error: Could not load transcript."

    # Split into chunks
    text_chunks = split_text(transcript, chunk_size)
    
    # Process each chunk
    summaries = []
    for chunk in text_chunks:
        prompt = f"""Summarize this part of the video transcript. Focus on the main points and key information:

{chunk}

Provide a clear and concise summary of this segment."""

        response = client.chat.completions.create(
            model="veera-test",
            messages=[
                {"role": "system", "content": "You are an expert at summarizing video transcripts. Create clear, coherent summaries that capture the main points."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.3
        )
        summaries.append(response.choices[0].message.content)

    # Combine chunk summaries
    combined_summaries = "\n\n".join(summaries)

    # Generate final summary
    final_response = client.chat.completions.create(
        model="veera-test",
        messages=[
            {"role": "system", "content": "Create a coherent, well-structured final summary from these segment summaries."},
            {"role": "user", "content": f"Create a final, coherent summary from these segment summaries:\n\n{combined_summaries}"}
        ],
        max_tokens=200,
        temperature=0.3
    )

    return final_response.choices[0].message.content

In [51]:
# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key="4d896b416db14383a7f1012bba28d751",
    api_version="2024-02-01",
    azure_endpoint="https://veeratest.openai.azure.com/"
)

if __name__ == "__main__":
    video_id = "PyZ0E7ukhBo"
    transcript = get_youtube_transcript_auto_generated(video_id)
    summary = get_video_summary(transcript, client)
    print("\nFinal Summary:")
    print(summary)

Fetching available transcripts for the video...
Hindi (auto-generated) True True
Auto-generated transcript found, language: hi

Final Summary:
The segment discusses the concept of "belly button oils" in Ayurveda and their benefits, as explained by the founders of Navel Sutra, Swati and Vaibhav. They emphasize the natural and beneficial aspects of their Ayurvedic belly button oils, formulated with the help of Ayurvedic doctors, and their aim to promote Ayurveda and Indian culture globally. The challenges and growth of their business are highlighted, emphasizing the need for credibility and trust in the Ayurvedic industry. They also share their journey of starting the business and the financial aspects, emphasizing the importance of utilizing skills and making decisions for the best outcomes.

The speaker discusses pricing strategies and suggests a marketing spend for a product priced at 900. They emphasize the importance of performance marketing and brand building, using the example of 